# 06-02-02

- ScalerMinMax
- Eucledian Distance
- Only Transition Matrices
- Remove Constant Features

In [1]:
import pandas as pd
from sklearn.cluster import KMeans
import seaborn as sns
from matplotlib import pyplot as plt 
import math
from ipynb.fs.full.Utils import createKDeplot, loadDataFrames, loadCoverGroups

import numpy as np
from sklearn.preprocessing import StandardScaler
from ipynb.fs.full.Pipeline import removeConstantFeatures, applyBoxCoxAllFeatures, centerAndScale, applyEuclideanDistance, getIndexAndVector, applyBoxCoxFeatures, scalerMinMax

from ipynb.fs.full.SearchInfo import searchByID, searchBySpotify, getTrackSpotifyInfo
from ipynb.fs.full.Utils import append_df_to_csv,transformToPercentage

In [2]:
cover = loadCoverGroups()
listCovers = cover[0]
originalSongId = listCovers[0]
coverSongId = listCovers[1]

In [3]:
cover

[[180849,
  6001027,
  1686718,
  5000013,
  5000001,
  5000007,
  5000008,
  5000023,
  5000017,
  5000019,
  5000020,
  5000018,
  5000011,
  5000012],
 [6001971, 6001984],
 [6002492, 6002491],
 [6000699, 6001165, 6000182],
 [6000104, 6000273],
 [6000768, 6000784],
 [6001931, 6001748],
 [6002043, 6002073],
 [6002244, 6000194],
 [6001209, 6000606],
 [6000352, 6001248],
 [6000549, 6001508, 6001242],
 [6001918, 6001997],
 [6001651, 6001900],
 [6000263, 6001570, 6002374],
 [6001822, 6001653],
 [6000763, 6000668],
 [6000764, 6000667],
 [6001543, 6001541],
 [6000762, 6000773],
 [6000771, 6000670],
 [6001747, 6001952],
 [6000548, 6002203],
 [6000172, 6002133],
 [6001762, 6001675],
 [6002147, 6002146],
 [6000767, 6000775],
 [6000766, 6000673],
 [6001506, 6000529],
 [6000397, 6000409],
 [6001914, 6002001],
 [6002482, 6002454],
 [6000765, 6000783],
 [6000761, 6000782],
 [6000814, 6000813],
 [6001224, 6000140]]

In [4]:
originalSongId


180849

In [5]:
spotifyFeatures, transitionMatrices, df = loadDataFrames()

In [6]:
ids = transitionMatrices.index.tolist()

In [7]:
transitionMatrices.sum(axis = 1).sum(axis=0)

5979.999999999996

In [8]:
transitionMatrices = removeConstantFeatures(transitionMatrices)

In [9]:
transitionMatrices = scalerMinMax(transitionMatrices)

In [10]:
transitionMatrices.describe()

,0,1,2,3,4,5,6,7,8,9,...,1027,1028,1029,1030,1031,1032,1033,1034,1035,1036
count,5980.000000,5980.000000,5980.000000,5980.000000,5980.000000,5980.000000,5980.000000,5980.000000,5980.000000,5980.000000,...,5980.000000,5980.000000,5980.000000,5980.000000,5980.000000,5980.000000,5980.000000,5980.000000,5980.000000,5980.000000
mean,0.028882,0.002190,0.000790,0.000797,0.001207,0.006105,0.022190,0.006024,0.000738,0.001866,...,0.000480,0.000834,0.001752,0.001237,0.000387,0.000167,0.002667,0.000716,0.000167,0.000992
std,0.072392,0.023851,0.018939,0.016458,0.024314,0.037906,0.083196,0.037311,0.019278,0.024825,...,0.018682,0.019811,0.023745,0.023198,0.014235,0.012932,0.026839,0.020779,0.012932,0.022374
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.021205,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


## Applying eucledian distance

In [11]:
def euclidean_distance(x, y):   
    return np.sqrt(np.sum((x - y) ** 2))

In [12]:
transitionMatrices

,0,1,2,3,4,5,6,7,8,9,...,1027,1028,1029,1030,1031,1032,1033,1034,1035,1036
0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0
1,0.219907,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0
2,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.126509,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0
3,0.034926,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0
4,0.016493,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.036458,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5975,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0
5976,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0
5977,0.000000,0.043956,0.0,0.073077,0.0,0.035897,0.086610,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0
5978,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.201058,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.238095,0.0,0.0,0.0,0.547619,0.0,0.0


In [13]:
transitionMatrices['id'] = ids
transitionMatrices = transitionMatrices.set_index('id')

In [14]:
transitionMatrices

,0,1,2,3,4,5,6,7,8,9,...,1027,1028,1029,1030,1031,1032,1033,1034,1035,1036
id,,,,,,,,,,,,,,,,,,,,,
23,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0
105,0.219907,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0
118,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.126509,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0
120,0.034926,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0
292,0.016493,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.036458,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6002496,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0
6002497,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0
6002498,0.000000,0.043956,0.0,0.073077,0.0,0.035897,0.086610,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0


In [15]:
originalSongVector = transitionMatrices.loc[[originalSongId]].iloc[0]

In [16]:
ranking = []
for index in range(0, len(transitionMatrices)):
    ranking.append([transitionMatrices.index[index], euclidean_distance(transitionMatrices.iloc[index, :], originalSongVector)])


In [17]:
ranksDF = pd.DataFrame(ranking)

In [18]:
ranksDF = ranksDF.rename(columns={0: "id", 1: "rank"})

In [19]:
ranksDF = ranksDF.sort_values(by="rank")

In [20]:
ranksDF.reset_index(drop=True, inplace=True)

In [21]:
ranksDF

,id,rank
0,180849,0.000000
1,1148352,0.396547
2,2846348,0.427019
3,1186397,0.462870
4,2065851,0.472398
...,...,...
5975,6002253,3.055102
5976,6001439,3.080084
5977,6002278,3.155543
5978,6002260,3.193442


In [22]:
ranksDF[ranksDF['id']==coverSongId]

,id,rank
73,6001027,0.618899


In [23]:
ranksDF[ranksDF['id'].isin(listCovers)]

,id,rank
0,180849,0.000000
21,5000018,0.545219
36,5000007,0.568267
39,5000019,0.573444
73,6001027,0.618899
96,1686718,0.637745
114,5000013,0.647101
117,5000011,0.648664
118,5000017,0.648733
189,5000012,0.690570


In [24]:
# To extract covers info change path in util if needed
ranks = transformToPercentage(ranksDF.head(10), '0202')
ranks

,id_chord,id_spotify_track,name,release_date,percentage,rank,position,id_experiment
0,180849,3jButwtJMLx3Ub61BfRiHh,the scientist,2022-06-03,100.000,0.000000,0,0202
1,1148352,2R6UrJ8uWbSIiHWmvRQvN8,whiskey in the jar,1998-01-01,99.603,0.396547,1,0202
2,2846348,4j5ffIFh7bFT7GZciP1TCy,10000 hours,2021-08-13,99.573,0.427019,2,0202
3,1186397,01uqI4H13Gsd8Lyl1EYd8H,same love,2012-10-09,99.537,0.462870,3,0202
4,2065851,43IbCqMcMvZi1v16LFAE3n,safe,2017-06-01,99.528,0.472398,4,0202
5,1909308,35r28RDot7nPE7y9K9H7l0,feeling whitney,2016-12-09,99.523,0.477496,5,0202
6,1108563,7f1mo7pArMj0b09g2N703b,charlie brown,2022-06-03,99.517,0.482608,6,0202
7,1168573,1NNAI51EuoRWw1ydX1zV7S,daylight,2012-01-01,99.516,0.484352,7,0202
8,3504560,5wANPM4fQCJwkGd4rN57mH,drivers license,2021-05-21,99.504,0.495691,8,0202
9,1065079,2U8g9wVcUu9wsg6i7sFSv8,every teardrop is a waterfall,2011-10-24,99.502,0.497568,9,0202
